The dataset for the assignment is a brain tumor data (brain tumor dataset.csv) that
has 20 columns and 20000 rows. The first objective is to classify the data based on tumor
type. You can use the 80:20 train-test split ratio.
(a) Build a Support vector machine classifier model to classify whether a tumor is
benign or malignant. [Marks:15]
(b) Plot the AUC-ROC curve for the model built. Also print the classification report.
[Marks:10]
(c) Use PCA to reduce the dimension to 2 features. Build a classifier model with this
data and plot the hyperplane and margins. [Marks:15]

In [2]:
from seaborn import load_dataset, pairplot
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import OneHotEncoder

from sklearn import datasets
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.datasets import make_classification

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv('./brain_tumor_dataset.csv')
df.head()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [4]:
df = df.drop('Patient_ID',axis=1)
df.head()

,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [5]:
df.isnull().sum()

Age                    0
Gender                 0
Tumor_Type             0
Tumor_Size             0
Location               0
Histology              0
Stage                  0
Symptom_1              0
Symptom_2              0
Symptom_3              0
Radiation_Treatment    0
Surgery_Performed      0
Chemotherapy           0
Survival_Rate          0
Tumor_Growth_Rate      0
Family_History         0
MRI_Result             0
Follow_Up_Required     0
dtype: int64

In [6]:
for i in df:
    print(df[i].unique())

[73 26 31 29 54 27 72 61 50 79 58 71 62 37 20 67 65 36 46 55 76 49 41 38
 63 44 56 70 42 64 48 52 78 23 39 33 32 34 30 25 75 43 47 28 60 66 35 57
 22 21 59 69 45 40 51 74 24 53 68 77]
['Male' 'Female']
['Malignant' 'Benign']
[5.37561155 4.84709779 5.58839071 ... 3.61863429 8.5190863  9.71676844]
['Temporal' 'Parietal' 'Frontal' 'Occipital']
['Astrocytoma' 'Glioblastoma' 'Meningioma' 'Medulloblastoma']
['III' 'II' 'I' 'IV']
['Vision Issues' 'Headache' 'Seizures' 'Nausea']
['Seizures' 'Headache' 'Vision Issues' 'Nausea']
['Seizures' 'Nausea' 'Headache' 'Vision Issues']
['No' 'Yes']
['No' 'Yes']
['No' 'Yes']
[51.31257903 46.37327271 47.07222097 ... 89.54380317 83.30678093
 47.43346759]
[0.11187562 2.16573562 1.88422778 ... 2.33288086 2.38720166 2.46407721]
['No' 'Yes']
['Positive' 'Negative']
['Yes' 'No']


In [7]:
df.shape

(20000, 18)

In [8]:
# dividing the columns into types 

binary_cols = ['Gender', 'Radiation_Treatment', 'Surgery_Performed', 
                'Chemotherapy', 'Family_History', 'MRI_Result', 'Follow_Up_Required']
categorical_cols = ['Location', 'Histology', 'Symptom_1', 'Symptom_2', 'Symptom_3']
numeric_cols = ['Age', 'Tumor_Size', 'Survival_Rate', 'Tumor_Growth_Rate']
ordinal_cols = ['Stage']

# encode Tumor_Type seperately in the end -> binaryencoding

# binary_cols --> BinaryEncoding 
# categorical_cols --> OneHotEncoding
# Stage --> OrdinalEncoding

In [ ]:
# Plotting the data to see if there is class imbalance 

# y.value_counts().plot.pie(autopct='%1.1f%%', startangle=90, cmap='coolwarm')
# plt.ylabel('')
# plt.legend()
# plt.title('Target Variable Distribution')
# plt.show()

KeyboardInterrupt: 

In [ ]:
# Create label encoder for binary columns
label_encoder = LabelEncoder()
onehot = OneHotEncoder(sparse_output=False)
ordinal = OrdinalEncoder(categories=[['I', 'II', 'III', 'IV']])


# Create a copy of X to avoid modifying the original dataframe
df_encoded = df.copy()

# Encode binary columns using LabelEncoder
for col in binary_cols:
    df_encoded[col] = label_encoder.fit_transform(df_encoded[col])

# Encode categorical columns using OneHotEncoder

categorical_encoded = onehot.fit_transform(df_encoded[categorical_cols])
categorical_feature_names = onehot.get_feature_names_out(categorical_cols)

# Encode ordinal column (Stage)

df_encoded[ordinal_cols] = ordinal.fit_transform(X_encoded[ordinal_cols])

# Convert categorical encoded array to dataframe
categorical_df = pd.DataFrame(categorical_encoded, columns=categorical_feature_names)

# Drop original categorical columns and concatenate with one-hot encoded columns
df_encoded = df_encoded.drop(columns=categorical_cols)
df_encoded = pd.concat([df_encoded, categorical_df], axis=1)

df_encoded.head()


In [ ]:
# pre-processing pipeline 

stage_categories = [['I', 'II', 'III', 'IV']]


# Create separate transformers for binary, categorical and ordinal columns
binary_transformer = OneHotEncoder(sparse_output=False)
categorical_transformer = OneHotEncoder(sparse_output=False)
ordinal_transformer = OrdinalEncoder(categories=stage_categories)

# Create the preprocessing pipeline with correct transformers
preprocess_pipeline = ColumnTransformer(
    transformers=[
        ('binary', binary_transformer, binary_cols),
        ('categorical', categorical_transformer, categorical_cols),
        ('ordinal', ordinal_transformer, ordinal_cols)
    ],
    remainder='passthrough'
)

# Transform the features
X_encoded = preprocess_pipeline.fit_transform(X)

# remainder='passthrough' --> unaffected columns will be kept as it is 
# columns transformer accepts list of encoders tuples ("random_name_of_encoder","actual encoder",list of column names)

In [ ]:
X=df.drop(['Tumor_Type'],axis=1)
y = df['Tumor_Type']

In [ ]:
X_encoded = preprocess_pipeline.fit_transform(X)

In [ ]:
X_encoded.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
# # Create separate transformers for binary, categorical and ordinal columns
# binary_transformer = OneHotEncoder(sparse_output=False)
# categorical_transformer = OneHotEncoder(sparse_output=False)
# ordinal_transformer = OrdinalEncoder(categories=stage_categories)

# # Create the preprocessing pipeline with correct transformers
# preprocess_pipeline = ColumnTransformer(
#     transformers=[
#         ('binary', binary_transformer, binary_cols),
#         ('categorical', categorical_transformer, categorical_cols),
#         ('ordinal', ordinal_transformer, ordinal_cols)
#     ],
#     remainder='passthrough'
# )

# # Transform the features
# X_encoded = preprocess_pipeline.fit_transform(X)

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=40)

X_train.shape , X_test.shape ,y_train.shape,y_test.shape , 



In [ ]:

# Creating the encoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)